In [2]:
# True : 문서의 임베딩을 피클 파일에서 읽음
# False : 임베딩을 새로함
is_use_embeddings_from_pickle = True

# 문서 임베딩의 피클 파일명
embeddings_pickle_filename = 'embeddings_text-embedding-3-large_dim3072.pkl'

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
 # 색인 대상 문서 및 평가 데이터 다운로드
 #!wget https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000291/data/data.tar.gz
 #!tar -xzvf data.tar.gz

## 환경 설정
검색엔진을 위한 Elasitcsearch, 임베딩 생성을 위한 sentence transformers, LLM 사용을 위한 openai client 설치가 필요합니다.

In [5]:
# GPU를 사용하는 버전의 Faiss Python 패키지 설치
#!pip install faiss-gpu

# OpenAI Python 패키지 설치
#!pip install openai==1.7.2

# 임베딩 생성을 위한 벡터 인코더 설치
#!pip install sentence-transformers==2.2.2

In [6]:
import os
import json
import faiss
import numpy as np

In [7]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = "Your API Key"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

In [8]:
# from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
#model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")      # TODO 임베딩 잘 만드는 모델을 찾아보자.
#model = SentenceTransformer("jhgan/ko-sroberta-multitask")


# SetntenceTransformer를 이용하여 임베딩 생성
# def get_embedding(sentences):
#     return model.encode(sentences)

def get_embedding(text, model="text-embedding-3-large"):
   return client.embeddings.create(input=[text], model=model).data[0].embedding

def texts_to_tensor(texts, model="text-embedding-3-large"):
    embeddings = [get_embedding(text, model) for text in texts]
    return embeddings


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        
        #embeddings = get_embedding(contents)
        embeddings = texts_to_tensor(contents)
        
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


In [9]:
# 문서의 content 필드에 대한 임베딩 생성
with open("../../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

In [10]:
import pickle

if not is_use_embeddings_from_pickle:
    embeddings = get_embeddings_in_batches(docs)
    
    # 나중을 위해 문서 임베딩을 피클 파일로 저장.
    with open(embeddings_pickle_filename, 'wb') as file:
        pickle.dump(embeddings, file)
else:
    # 피클 파일로부터 embeddings 리스트 복원
    with open(embeddings_pickle_filename, 'rb') as file:
        embeddings = pickle.load(file)

In [11]:
print(f'embedding dimension : {len(embeddings[0])}')
print(f'embeddings type : {type(embeddings)}')
print(f'embeddings length : {len(embeddings)}')
print(f'embeddings[0] : {embeddings[0]}')

embedding dimension : 3072
embeddings type : <class 'list'>
embeddings length : 4272
embeddings[0] : [0.03257926180958748, -0.008541478775441647, -0.00882707629352808, -0.01854267716407776, -0.011275810189545155, 0.015464572235941887, -0.016226164996623993, -0.019177338108420372, 0.011053678579628468, -0.07544003427028656, 0.036768026649951935, -0.008287614211440086, 0.014840489253401756, -0.00774815259501338, 0.0235670767724514, -0.011244077235460281, 0.01890231855213642, -0.007134647108614445, 0.04366467520594597, 0.0019330715294927359, -0.001348654506728053, -0.03587950021028519, 0.004318338818848133, 0.0014755867887288332, -0.013698099181056023, -0.005405195988714695, 0.014353915117681026, 0.01808783784508705, 0.00666922889649868, -0.039116270840168, -0.029025161638855934, 0.0273327324539423, -0.02068994753062725, -0.002837463514879346, -0.031986914575099945, -0.024476757273077965, 0.01512608677148819, 0.00987955555319786, 0.0035514570772647858, 0.02659229375422001, -0.038164280354

In [12]:
embedding_dims = len(embeddings[0])

index_docs = faiss.IndexFlatL2(embedding_dims)
index_docs.add(np.array(embeddings).astype('float32'))

In [13]:

# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_emb = get_embedding(query_str)
    query_emb = np.array([query_emb]).astype('float32')

    # Faiss 인덱스 생성시 IndexFlatL2() 함수를 사용했으므로 scores 의 값이 작을수록 관련성이 높음!
    # (L2 는 유클리드 거리를 의미하므로)
    scores, offsets = index_docs.search(query_emb, size)
    
    ret = [
            {
                "score": scores[0][i].astype(float),    # 이렇게 타입을 바꾸지 않으면 나중에 json dump 시 에러 발생함.
                "doc": docs[offsets[0][i]] 
            }
            for i in range(len(scores[0]))
        ]

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return ret

In [14]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
query_str = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"
ret = dense_retrieve(query_str, 5)
print(ret)

[{'score': 0.49018144607543945, 'doc': {'docid': '464ace62-ddf2-423d-a5d7-2f17e6785c8e', 'src': 'ko_ai2_arc__ARC_Challenge__train', 'content': '금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.'}}, {'score': 0.8304318785667419, 'doc': {'docid': '45b8eb6a-87e3-4333-b01b-7c8b772f827f', 'src': 'ko_mmlu__astronomy__validation', 'content': '금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.'}}, {'score': 0.8955973982810974, 'doc': {'docid': 'da6c8a3f-45a9-4025-a63a-47c05ba2b336', 'src': 'ko_mmlu__astronomy__test', 'content':

## RAG 구현

준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [15]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- search api를 호출할 때, 사용자의 대화가 멀티턴이 아니라면, 사용자의 대화 내용을 그대로 standalone_query 에 대입한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [16]:
from sentence_transformers import CrossEncoder
cross_encoder_model = CrossEncoder("bongsoo/albert-small-kor-cross-encoder-v1")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [17]:
def merge_retrieve_result_from_elastic(standalone_query, result1, result2):
    hits_merged_list = []
    
    def append_without_duplicate(result):
        for rst in result['hits']['hits']:
            # 중복해서 추가되지 않도록.
            is_found = False
            
            for rst2 in hits_merged_list:
                if rst['_source']['docid'] == rst2['_source']['docid']:
                    is_found = True
                    break
            
            if is_found: continue
            
            hits_merged_list.append(rst)
    
    # 중복 없이 hits_merged_list 에 추가
    #append_without_duplicate(result1)
    append_without_duplicate(result2)  
    
    # 유사도 비교를 할 리스트 만들기
    # pair_list = []    
    # for rst in hits_merged_list:
    #     pair_list.append( (standalone_query, rst['_source']['content']) )
    
    # # 유사도 계산
    # scores = cross_encoder_model.predict(pair_list)
    
    # # 내림차순 정렬을 위해 유사도 리스트에 인덱스 추가
    # scores_with_idx = []
    # for i, score in enumerate(scores):
    #     dic = {}
    #     dic['idx'] = i
    #     dic['score'] = score
    #     scores_with_idx.append(dic)
    
    # # score 기준으로 내림차순 정렬
    # scores_with_idx = sorted(scores_with_idx, key=lambda x: x['score'], reverse=True)
    
    # # 최종 결과 만들기
    # ret = []
    # for dic in scores_with_idx:
    #     ret.append(hits_merged_list[dic['idx']])
    
    # return ret
    return hits_merged_list

In [18]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")
        response["standalone_query"] = standalone_query

        #search_result_sparse = sparse_retrieve(standalone_query, 3)
        search_result_dense = dense_retrieve(standalone_query, 3)
        
        #hits_merged_list = merge_retrieve_result_from_elastic(standalone_query, search_result_sparse, search_result_dense)
        hits_merged_list = search_result_dense
        
        retrieved_context = []
        for i, rst in enumerate(hits_merged_list):
            if i >= 3:
                break   # 최대 3개까지만..
            
            retrieved_context.append(rst["doc"]["content"])
            
            response["topk"].append(rst["doc"]["docid"])
            response["references"].append({"score": rst["score"], "content": rst["doc"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response

In [19]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 0:
            #     break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [20]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
submission_filename = "submission_008.csv"

eval_rag("../../data/eval.jsonl", submission_filename)

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: 나무의 분류를 조사하기 위한 방법은 여러 가지가 있습니다. 먼저, 나무의 잎 모양, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 나무를 분류할 수 있습니다. 또한, 식물학적 분류학적인 정보를 통해 나무를 식별할 수도 있습니다. 이러한 방법들을 통해 나무를 분류하고자 할 때는 관찰과 분류에 대한 기본적인 지식이 필요합니다. 이러한 방법들은 나무의 특징과 관련된 정보를 활용하여 분류하는 것이기 때문에, 충분한 정보가 없을 경우 분류를 할 수 없다고 말씀드립니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 죄송합니다, 현재 제가 찾은 정보로는 각 나라의 공교육 지출 현황에 대한 구체적인 정보를 제공할 수 없습니다. 더 많은 정보를 찾아보고 싶으시다면 해당 국가의 교육부나 통계청 웹사이트를 참고하시는 것이 도움이 될 것입니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 기억 상실증은 다양한 원인에 의해 발생할 수 있습니다. 주로 인지 기능의 손상으로 인해 발생하며, 인지 기능은 인지, 기억, 판단, 추론 등 다양한 기능을 포함합니다. 기억 상실증은 인지 기능의 손상으로 인해 기억력, 학습능력, 사고능력 등이 저하되는 증상을 보입니다. 또한, 스트레스, 외상, 뇌졸중, 알츠하이머병 등 다양한 요인으로 인해 발생할 수 있습니다. 따라서, 기억 상실증의 원인은 다양하며, 각각의 경우에 따라 적절한 치료와 관리가 필요합니다.

Test 3
Quest

#Reference

## Required Package

openai==1.7.2 <br>
elasticsearch==8.8.0 <br>
sentence_transformers==2.2.2 <br>



## 콘텐츠 라이선스

저작권 : <font color='blue'> <b> ©2023 by Upstage X fastcampus Co., Ltd. All rights reserved.</font></b>

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 업스테이지 및 패스트캠퍼스에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. </b>